In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle
import shutil

In [2]:
import yaml

with open('config_test.yaml') as f:
    conf = yaml.full_load(f)


In [3]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
global cuda_dev
cuda_dev = conf['trn_args']['device_id']

device = torch.device(cuda_dev if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:1


In [4]:
len(string.printable)
## train args
weight_dir=conf['dataset']['weight_path']
data_mode = 'label'

In [5]:
##decoding

all_letters = string.printable
n_letters = len(all_letters)
def letterToIndex(letter):
    return all_letters.find(letter)
def linesToTensor(lines):
    char_length = 7000

    tensor_one_hot = torch.zeros(len(lines), char_length, n_letters)
    tensor_label = torch.zeros(len(lines),char_length,1) ##line = batchsize
    
    for b, line in enumerate(lines): 
        len_line = min(char_length,len(line))
        line = line[:len_line]
        for li, letter in enumerate(line):
            tensor_one_hot[b][li + char_length - (len_line)][letterToIndex(letter)] = 1 #뒤로 맞춰줌 batch*character*100
            tensor_label[b][li + char_length - (len_line)][0] =letterToIndex(letter) #batch*character*1

    return tensor_one_hot,tensor_label

In [6]:

global input_size,hidden_size,num_layers
hidden_size=conf['trn_args']['hidden_size']
num_layers=conf['trn_args']['num_layer']
class LangModel(nn.Module):

    def __init__(self, preTrained='True', input=100):
        super(LangModel, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, hidden_size, num_layers, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(hidden_size, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.to(device)
        h0 = ( Variable(torch.zeros(num_layers, text.size(0), hidden_size)).to(device),  Variable(torch.zeros(num_layers, text.size(0), hidden_size)).to(device))

        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred,lang_feature


In [7]:
import torch.utils.data as data

class chat_ds(data.Dataset):
    def __init__(self,d_type):
        self.gt_range =  1-conf['trn_args']['hl_range']
        self.d_type=d_type
        with open(conf['dataset']['label_path'],'rb') as f1:  
            self.gt=pickle.load(f1)
        with open(conf['dataset']['chat_path'],'rb') as f2:  
            self.text=pickle.load(f2)
        #self.sample=list(map(int,list(self.gt.keys())))
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        if d_type =='total':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654'] + ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630'] + ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.gt[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)

        begin_pos = 0 
        hl_frames = []
        for it, cur_pos in enumerate(pos_idx):
            if it+1 < len(pos_idx): 
                if((pos_idx[it+1] - cur_pos) > 1):#cur_pos와 cur_pos+1 간격이 1보다 크면, 즉 다른 구간이면
                    begin = int((it+1 - begin_pos) * self.gt_range) + begin_pos
                    hl_frames.extend( pos_idx[begin: it] ) #한구간의 하이라이트 25%만 사용하겠다.
                    begin_pos = it+1


        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[hl_frames] = len(sampling) / float(len(hl_frames))
        self.WeightedSampling = sampling

        self.sum=np.insert(np.cumsum([len(self.gt[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)
            vid = np.where(vid[0]>0)[0][0]
            vframe=index-self.sum[vid]
            game_id=str(self.sample[vid])


            win_text=''
            for idx in range(conf['trn_args']['window_size']): #7 : window size
                if vframe+idx<len(self.text[str(game_id)]):
                    win_text+=self.text[str(game_id)][vframe+idx][:1000]+'\n'

            label=self.gt[str(game_id)][vframe]
            return win_text,label,str(game_id)
        
            
            



In [8]:
class SampleSequentialSampler(sampler.Sampler):
    """Samples elements sequentially, always in the same order.
    Arguments:
        data_source (Dataset): dataset to sample from
        offset (int): offset between the samples
    """

    def __init__(self, data_source, offset=10):
        self.num_samples = len(data_source) 
        self.offset = offset

    def __iter__(self):
        return iter(np.arange(0, self.num_samples, self.offset ))

    def __len__(self):
        return len(np.arange(0, self.num_samples, self.offset ))


In [9]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [10]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [11]:
#game_id='102844212428895431'
train=chat_ds('train')
val=chat_ds('val')

data load fin
data load fin


In [12]:
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=conf['trn_args']['trn_bs'],sampler=sampler1)
# sampler2 =  SampleSequentialSampler(val, 30)
val_loader=torch.utils.data.DataLoader(val,batch_size=conf['trn_args']['trn_bs'])

In [13]:
###### model load #####
if data_mode == 'label':
    input_val = 1
else:
    inputs_val = 100
model=LangModel(input=input_val).to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [14]:
weight_dir=conf['dataset']['weight_path']


In [16]:
#test

test=chat_ds('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=1024)
dataset=weight_dir+'train_best'
checkpoint=torch.load(dataset,map_location='cuda:1')
model.load_state_dict(checkpoint)
model.to(device)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (text,labels,g) in enumerate(test_loader):
        if data_mode == 'label':
            inputs=linesToTensor(text)[1]
        else:
            inputs=linesToTensor(text)[0]
        inputs = inputs.to(device)
        labels = labels.to(device)
        output_p,output_f=model(inputs)
        
        
        print(output_p)

        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output_p.cpu(),labels.cpu())
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output_p)
        for idx,gi in enumerate(g):
            if gi not in result.keys():
                result[gi]=pred[idx].tolist()
            else:
                result[gi]+=pred[idx].tolist()
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
tensor([[ 0.6615, -0.6501],
        [ 0.7244, -0.7003],
        [ 0.7247, -0.7524],
        ...,
        [-0.3310,  0.2938],
        [-0.0055,  0.0037],
        [-0.4293,  0.3347]], device='cuda:1')
35 256 667 66 tensor(291) tensor(101)
tensor([[-0.5311,  0.4072],
        [-0.8501,  0.7421],
        [-0.8436,  0.6959],
        ...,
        [-0.7389,  0.6274],
        [-0.2710,  0.2499],
        [-0.2352,  0.1562]], device='cuda:1')
109 384 487 44 tensor(493) tensor(153)
tensor([[-1.0422,  0.9344],
        [ 0.1201, -0.1587],
        [ 0.6692, -0.6609],
        ...,
        [ 0.4201, -0.5264],
        [ 0.5620, -0.5660],
        [ 0.6489, -0.6481]], device='cuda:1')
76 177 703 68 tensor(253) tensor(144)
tensor([[ 0.4906, -0.5920],
        [ 0.7379, -0.7733],
        [ 0.6286, -0.6296],
        ...,
        [-0.9287,  0.7963],
        [-0.6596,  0.5851],
        [-0.4784,  0.4389]], device='cuda:1')
63 235 697 29 tensor(298) tensor(92)
tensor([[ 0.2691, -0.2997],
        [ 

In [17]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])